In [1]:
import pandas as pd
import pprint as pp
import optuna

import torch
import torchtext

from sklearn.model_selection import train_test_split

from preprocessing_utils import preprocess_data_for_nn

from torch_nn_model_2 import (
    get_best_available_device,
    train_nn_model,
    NNHyperparams,
)

torchtext.disable_torchtext_deprecation_warning()

c:\Users\david\miniconda3\envs\python311\Lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\david\miniconda3\envs\python311\Lib\site-packages\torchtext\utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\david\miniconda3\envs\python311\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable t

In [2]:
def load_dataset(
    sequence_cols: list[str],
    char_cols: list[str],
    vector_cols: list[str],
    scalar_cols: list[str],
    vector_dims: dict[str, int],
    load_fresh: bool = False,
    sample_size: int | None = None,
):
    """
    Load the dataset, optionally reloading it fresh.
    """
    if load_fresh:
        print("Loading dataset fresh...")
        df = pd.read_csv("dataset/apk_analysis_dataset.csv")

        if sample_size is not None:
            print(f"Sampling {sample_size} rows from the dataset...")
            df = df.sample(sample_size, random_state=42)

        df, vocab_dict = preprocess_data_for_nn(
            df,
            sequence_cols=sequence_cols,
            char_cols=char_cols,
            vector_cols=vector_cols,
            scalar_cols=scalar_cols,
            vector_dims=vector_dims,
        )

        print("Saving preprocessed dataset and vocab_dict...")

        df.to_pickle("dataset/processed_apk_analysis_dataset.pkl")
        torch.save(vocab_dict, "dataset/processed_vocab_dict.pth")

        print("Preprocessing complete and saved.")

    else:
        print("Loading last preprocessed dataset...")
        df = pd.read_pickle("dataset/processed_apk_analysis_dataset.pkl")
        vocab_dict = torch.load("dataset/processed_vocab_dict.pth")

    return df, vocab_dict

In [3]:
SEQUENCE_COLS = [
    "activities_list",
    "services_list",
    "receivers_list",
    "permissions_list",
    "api_calls_list",
]

CHAR_COLS = ["fuzzy_hash"]
VECTOR_COLS = ["opcode_counts"]
SCALAR_COLS = ["file_size"]
VECTOR_DIMS = {"opcode_counts": 768}

# Load dataset
df, vocab_dict = load_dataset(
    SEQUENCE_COLS,
    CHAR_COLS,
    VECTOR_COLS,
    SCALAR_COLS,
    VECTOR_DIMS,
    load_fresh=False,
    sample_size=None,
)

df, df_sample = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df["is_malware"]
)

device = get_best_available_device()

Loading last preprocessed dataset...
Using CUDA device: NVIDIA GeForce RTX 4070 SUPER



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\david\miniconda3\envs\python311\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\david\miniconda3\envs\python311\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\david\miniconda3\envs\python311\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start(

In [4]:
display(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 15977 entries, 19862 to 3961
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   file_size         15977 non-null  float64
 1   fuzzy_hash        15977 non-null  object 
 2   activities_list   15977 non-null  object 
 3   services_list     15977 non-null  object 
 4   receivers_list    15977 non-null  object 
 5   permissions_list  15977 non-null  object 
 6   api_calls_list    15977 non-null  object 
 7   opcode_counts     15977 non-null  object 
 8   is_malware        15977 non-null  int64  
dtypes: float64(1), int64(1), object(7)
memory usage: 1.2+ MB


None

,file_size,fuzzy_hash,activities_list,services_list,receivers_list,permissions_list,api_calls_list,opcode_counts,is_malware
19862,4939479.0,"[47, 64, 18, 17, 62, 19, 63, 63, 29, 34, 31, 4...","[238986, 10, 238988, 238983, 3, 238985, 4, 22,...","[9, 4, 13, 5, 6, 8, 11, 3, 12, 10, 20857, 0, 0...","[4, 8, 13, 16, 11, 7, 12, 6, 10, 5, 9, 0, 0, 0...","[20, 3, 23, 17, 4, 5, 11, 41, 0, 0, 0, 0, 0, 0...","[175378, 2972, 4510, 6430, 5098, 16823, 16449,...","[268.0, 2926.0, 3840.0, 0.0, 700.0, 619.0, 0.0...",0
7826,4560508.0,"[36, 17, 10, 59, 6, 65, 21, 62, 17, 39, 62, 14...","[179512, 179513, 179511, 18, 179509, 179510, 1...","[20, 25, 26, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[18, 17, 11402, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[13, 9, 7, 24, 46, 21, 19, 4, 12, 8, 6, 22, 5,...","[1012942, 1012943, 1012944, 1012945, 1013007, ...","[164.0, 2642.0, 44.0, 0.0, 49.0, 10.0, 0.0, 32...",1
18535,4335600.0,"[53, 46, 15, 50, 10, 22, 23, 21, 21, 34, 50, 3...","[97608, 44202, 97613, 4, 97614, 22, 97609, 3, ...","[13, 10, 12, 11, 4, 5, 9, 0, 0, 0, 0, 0, 0, 0,...","[6, 13, 16, 12, 4, 9, 7, 10, 8, 11, 0, 0, 0, 0...","[14, 7, 18, 17, 5, 3, 4, 41, 12, 13, 23, 10, 3...","[83888, 197158, 2972, 16823, 16449, 16835, 167...","[389.0, 2931.0, 5771.0, 0.0, 532.0, 537.0, 0.0...",0
7789,2120068.0,"[56, 12, 11, 7, 57, 50, 12, 51, 34, 17, 45, 16...","[194481, 12198, 12199, 23835, 23836, 0, 0, 0, ...","[2126, 2127, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[22, 163, 11, 4, 182, 7, 21, 8, 3, 9, 15, 6, 2...","[302565, 121966, 302566, 418308, 302567, 41845...","[139.0, 3710.0, 541.0, 0.0, 113.0, 53.0, 0.0, ...",1
2356,269692.0,"[3, 30, 28, 62, 49, 62, 38, 33, 17, 45, 36, 62...","[721, 931, 707, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[212, 214, 213, 211, 284, 0, 0, 0, 0, 0, 0, 0,...","[153, 126, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[6, 9, 15, 3, 4, 19, 31, 10, 28, 62, 0, 0, 0, ...","[18370, 58927, 73, 84, 685, 731, 277, 592, 223...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, ...",1


In [5]:
# Make sure df_processed, vocab_dict, SEQUENCE_COLS, etc. are defined in your notebook scope
# df_processed = df # Or however you get your fully preprocessed DataFrame for the NN
# device = get_best_available_device() # Already defined


def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for Optuna to optimize.
    A trial will suggest a set of hyperparameters, train the model, and return a score.
    """
    # Define the search space for hyperparameters
    # These are examples, adjust ranges and types based on your intuition/needs
    lr = trial.suggest_float("max_learning_rate", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    embedding_dim = trial.suggest_categorical("embedding_dim", [64, 128, 256])

    hidden_dims_one_layer = [(16,), (32,), (64,), (128,), (256,)]
    hidden_dims_two_layers = [
        (256, 128),
        (256, 64),
        (256, 32),
        (256, 16),
        (128, 64),
        (128, 32),
        (128, 16),
        (64, 32),
        (64, 16),
        (32, 16),
    ]
    hiddem_dims_possible = hidden_dims_one_layer + hidden_dims_two_layers
    
    hidden_dims_idx = trial.suggest_int("hidden_dims_idx", 0, len(hiddem_dims_possible) - 1)
    hidden_dims = list(hiddem_dims_possible[hidden_dims_idx])
    print(f"Selected hidden_dims: {hidden_dims} (index {hidden_dims_idx})")

    dropout = trial.suggest_float("dropout", 0.1, 0.5, step=0.1)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)

    # Create NNHyperparams instance with suggested values
    # Keep other hyperparams fixed or include them in the trial if you want to tune them
    current_hyperparams = NNHyperparams(
        batch_size=batch_size,
        max_learning_rate=lr,
        epochs=5,
        early_stopping=True,
        patience=2,
        optimizer="adamw",  # Consider fixing or tuning
        weight_decay=weight_decay,
        embedding_dim=embedding_dim,
        hidden_dims=hidden_dims,
        dropout=dropout,
        seq_pooling="mean",
        n_classes=2,
        label_col="is_malware",
        dataloader_num_workers=2,
        dataloader_pin_memory=True,
        dataloader_persistent_workers=True,
        grad_scaler_max_norm=1.0,
    )

    print(f"\n--- Optuna Trial ---")
    print(f"Suggested Hyperparams: {current_hyperparams}")

    trained_model, results, fitted_scalers = train_nn_model(
        df=df_sample,  # Use your full preprocessed DataFrame or a sample
        vocab_dict=vocab_dict,
        sequence_cols=SEQUENCE_COLS,
        scalar_cols=SCALAR_COLS,
        char_cols=CHAR_COLS,
        vector_cols=VECTOR_COLS,
        vector_dims=VECTOR_DIMS,
        hyperparams=current_hyperparams,
        device=device,
        train_split_ratio=0.5,
        scoring_metric="f1",
        random_seed=42,
    )

    # unload the model to free memory
    del trained_model
    torch.cuda.empty_cache()

    import gc

    gc.collect()

    # Get the metric to optimize from the results
    # Ensure this matches the 'scoring_metric' used in train_nn_model
    # and the direction of optimization in optuna.create_study
    metric_to_optimize = results["final_metrics_best_model"].get("f1", 0.0)

    print(f"Trial finished. Validation Recall: {metric_to_optimize:.4f}")

    # Optuna tries to maximize the returned value by default if direction='maximize'
    return metric_to_optimize

In [6]:
# Create a study object. Define the direction of optimization.
# For metrics like F1, Recall, Precision, Accuracy, ROC AUC, PR AUC, you want to "maximize".
# If you were optimizing loss, you would use "minimize".
study = optuna.create_study(direction="maximize", study_name="debrim_nn_hyperopt")

# Start the optimization. n_trials is the number of different hyperparameter sets to try.
# Start with a small number (e.g., 10-20) to test, then increase for a more thorough search.
n_trials = 100
study.optimize(objective, n_trials=n_trials)

# Print the best hyperparameters found
print("\n--- Optuna Study Complete ---")
print(f"Number of finished trials: {len(study.trials)}")
print("Best trial:")
best_trial = study.best_trial

print(f"  Value (F1 Score): {best_trial.value:.4f}")
print("  Params: ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

# You can now use best_trial.params to configure your NNHyperparams
# for a final training run or for cross-validation.
best_hyperparams_dict = best_trial.params

hidden_dims_one_layer = [(16,), (32,), (64,), (128,), (256,)]
hidden_dims_two_layers = [
    (256, 128),
    (256, 64),
    (256, 32),
    (256, 16),
    (128, 64),
    (128, 32),
    (128, 16),
    (64, 32),
    (64, 16),
    (32, 16),
]
hiddem_dims_possible = hidden_dims_one_layer + hidden_dims_two_layers

# Extract all of the trials and their hyperparameters and show them
print("\nAll trials and their hyperparameters:")
all_trials = study.trials
for trial in all_trials:
    print(f"Trial {trial.number}: Value = {trial.value}, Params = {trial.params}")

print("\nTrials df:")
all_trails_df = study.trials_dataframe()
all_trails_df.to_csv("dataset/optuna_trials_debrim_nn.csv", index=False)
display(all_trails_df)

final_nn_hyperparams = NNHyperparams(
    batch_size=best_hyperparams_dict["batch_size"],
    max_learning_rate=best_hyperparams_dict["max_learning_rate"],
    epochs=20,
    early_stopping=True,
    patience=5,
    optimizer="adamw",
    weight_decay=best_hyperparams_dict["weight_decay"],
    embedding_dim=best_hyperparams_dict["embedding_dim"],
    hidden_dims=hiddem_dims_possible[best_hyperparams_dict["hidden_dims_idx"]],
    dropout=best_hyperparams_dict["dropout"],
    seq_pooling="mean",
    label_col="is_malware",
    n_classes=2,
    dataloader_num_workers=0,
    dataloader_pin_memory=True,
    dataloader_persistent_workers=False,
    grad_scaler_max_norm=1.0,
)

print("\nBest NNHyperparams based on Optuna search:")
pp.pprint(final_nn_hyperparams)

# Now you can use final_nn_hyperparams to train your model, perhaps with cross_val_train_nn_model
# nn_results, best_nn_model = cross_val_train_nn_model(
#     df=df, # Use your full preprocessed DataFrame
#     vocab_dict=vocab_dict,
#     sequence_cols=SEQUENCE_COLS,
#     scalar_cols=SCALAR_COLS,
#     char_cols=CHAR_COLS,
#     vector_cols=VECTOR_COLS,
#     vector_dims=VECTOR_DIMS,
#     hyperparams=final_nn_hyperparams, # Use the tuned hyperparameters
#     n_folds=5, # Example: 5 folds for final evaluation
#     n_repetitions=1,
#     scoring_metric="f1", # Or your preferred final metric
#     device=device,
#     random_seed=42,
# )

[I 2025-06-22 10:21:52,274] A new study created in memory with name: debrim_nn_hyperopt


Selected hidden_dims: [128, 32] (index 10)

--- Optuna Trial ---
Suggested Hyperparams: NNHyperparams(batch_size=16, max_learning_rate=0.0023856730313390243, epochs=5, early_stopping=True, patience=2, optimizer='adamw', weight_decay=1.6634066787303807e-05, embedding_dim=128, hidden_dims=[128, 32], dropout=0.4, seq_pooling='mean', n_classes=2, label_col='is_malware', dataloader_num_workers=2, dataloader_pin_memory=True, dataloader_persistent_workers=True, grad_scaler_max_norm=1.0)
Using device: cuda
Using f1 as the primary scoring metric for validation.
Performing internal train/validation split with ratio: 0.5
Training set size: 1997, Validation set size: 1998
Training set class distribution: {0: 1000, 1: 997}
Validation set class distribution: {0: 1000, 1: 998}
Using class weights: [0.9985     1.00150451]
Starting training...
Epoch 1, Batch 0/125, Train Loss: 0.7262, LR: 9.56e-05
Epoch 1, Batch 26/125, Train Loss: 0.6629, LR: 2.12e-04
Epoch 1, Batch 52/125, Train Loss: 0.5130, LR: 5.2

[I 2025-06-22 10:23:35,920] Trial 0 finished with value: 0.9674465920651069 and parameters: {'max_learning_rate': 0.0023856730313390243, 'batch_size': 16, 'embedding_dim': 128, 'hidden_dims_idx': 10, 'dropout': 0.4, 'weight_decay': 1.6634066787303807e-05}. Best is trial 0 with value: 0.9674465920651069.


Trial finished. Validation Recall: 0.9674
Selected hidden_dims: [32, 16] (index 14)

--- Optuna Trial ---
Suggested Hyperparams: NNHyperparams(batch_size=32, max_learning_rate=3.0173055455242765e-05, epochs=5, early_stopping=True, patience=2, optimizer='adamw', weight_decay=0.0003004062167267393, embedding_dim=64, hidden_dims=[32, 16], dropout=0.30000000000000004, seq_pooling='mean', n_classes=2, label_col='is_malware', dataloader_num_workers=2, dataloader_pin_memory=True, dataloader_persistent_workers=True, grad_scaler_max_norm=1.0)
Using device: cuda
Using f1 as the primary scoring metric for validation.
Performing internal train/validation split with ratio: 0.5
Training set size: 1997, Validation set size: 1998
Training set class distribution: {0: 1000, 1: 997}
Validation set class distribution: {0: 1000, 1: 998}
Using class weights: [0.9985     1.00150451]
Starting training...
Epoch 1, Batch 0/63, Train Loss: 0.7047, LR: 1.22e-06
Epoch 1, Batch 13/63, Train Loss: 0.6871, LR: 2.78e-

[I 2025-06-22 10:24:30,653] Trial 1 finished with value: 0.8289085545722714 and parameters: {'max_learning_rate': 3.0173055455242765e-05, 'batch_size': 32, 'embedding_dim': 64, 'hidden_dims_idx': 14, 'dropout': 0.30000000000000004, 'weight_decay': 0.0003004062167267393}. Best is trial 0 with value: 0.9674465920651069.


...

Trial finished. Validation Recall: 0.9724
Selected hidden_dims: [64] (index 2)

--- Optuna Trial ---
Suggested Hyperparams: NNHyperparams(batch_size=64, max_learning_rate=0.003279620690106178, epochs=5, early_stopping=True, patience=2, optimizer='adamw', weight_decay=8.308178047031056e-05, embedding_dim=64, hidden_dims=[64], dropout=0.5, seq_pooling='mean', n_classes=2, label_col='is_malware', dataloader_num_workers=2, dataloader_pin_memory=True, dataloader_persistent_workers=True, grad_scaler_max_norm=1.0)
Using device: cuda
Using f1 as the primary scoring metric for validation.
Performing internal train/validation split with ratio: 0.5
Training set size: 1997, Validation set size: 1998
Training set class distribution: {0: 1000, 1: 997}
Validation set class distribution: {0: 1000, 1: 998}
Using class weights: [0.9985     1.00150451]
Starting training...
Epoch 1, Batch 0/32, Train Loss: 0.6865, LR: 1.35e-04
Epoch 1, Batch 7/32, Train Loss: 0.6606, LR: 3.51e-04
Epoch 1, Batch 14/32, Tra

[I 2025-06-22 15:12:04,295] Trial 99 finished with value: 0.9723756906077348 and parameters: {'max_learning_rate': 0.003279620690106178, 'batch_size': 64, 'embedding_dim': 64, 'hidden_dims_idx': 2, 'dropout': 0.5, 'weight_decay': 8.308178047031056e-05}. Best is trial 80 with value: 0.9734335839598998.


Trial finished. Validation Recall: 0.9724

--- Optuna Study Complete ---
Number of finished trials: 100
Best trial:
  Value (F1 Score): 0.9734
  Params: 
    max_learning_rate: 0.006311264200084833
    batch_size: 64
    embedding_dim: 64
    hidden_dims_idx: 3
    dropout: 0.5
    weight_decay: 8.413267202596088e-05

All trials and their hyperparameters:
Trial 0: Value = 0.9674465920651069, Params = {'max_learning_rate': 0.0023856730313390243, 'batch_size': 16, 'embedding_dim': 128, 'hidden_dims_idx': 10, 'dropout': 0.4, 'weight_decay': 1.6634066787303807e-05}
Trial 1: Value = 0.8289085545722714, Params = {'max_learning_rate': 3.0173055455242765e-05, 'batch_size': 32, 'embedding_dim': 64, 'hidden_dims_idx': 14, 'dropout': 0.30000000000000004, 'weight_decay': 0.0003004062167267393}
Trial 2: Value = 0.9685785536159601, Params = {'max_learning_rate': 0.00047686427118995696, 'batch_size': 16, 'embedding_dim': 256, 'hidden_dims_idx': 14, 'dropout': 0.1, 'weight_decay': 0.000709815508396805

,number,value,datetime_start,datetime_complete,duration,params_batch_size,params_dropout,params_embedding_dim,params_hidden_dims_idx,params_max_learning_rate,params_weight_decay,state
0,0,0.967447,2025-06-22 10:21:52.275166,2025-06-22 10:23:35.920028,0 days 00:01:43.644862,16,0.4,128,10,0.002386,0.000017,COMPLETE
1,1,0.828909,2025-06-22 10:23:35.920028,2025-06-22 10:24:30.653731,0 days 00:00:54.733703,32,0.3,64,14,0.000030,0.000300,COMPLETE
2,2,0.968579,2025-06-22 10:24:30.653731,2025-06-22 10:40:31.917986,0 days 00:16:01.264255,16,0.1,256,14,0.000477,0.000710,COMPLETE
3,3,0.927717,2025-06-22 10:40:31.918986,2025-06-22 10:41:50.316013,0 days 00:01:18.397027,32,0.4,128,2,0.000086,0.000049,COMPLETE
4,4,0.962038,2025-06-22 10:41:50.317013,2025-06-22 10:48:57.538700,0 days 00:07:07.221687,64,0.2,256,14,0.000549,0.000031,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.967579,2025-06-22 15:07:51.722563,2025-06-22 15:08:32.244461,0 days 00:00:40.521898,64,0.5,64,3,0.005664,0.000226,COMPLETE
96,96,0.961538,2025-06-22 15:08:32.245964,2025-06-22 15:09:47.502461,0 days 00:01:15.256497,64,0.5,128,4,0.004806,0.000102,COMPLETE
97,97,0.969758,2025-06-22 15:09:47.503461,2025-06-22 15:10:33.159978,0 days 00:00:45.656517,64,0.4,64,3,0.002663,0.000143,COMPLETE
98,98,0.972376,2025-06-22 15:10:33.160982,2025-06-22 15:11:18.486789,0 days 00:00:45.325807,64,0.5,64,2,0.003400,0.000083,COMPLETE



Best NNHyperparams based on Optuna search:
NNHyperparams(batch_size=64,
              max_learning_rate=0.006311264200084833,
              epochs=20,
              early_stopping=True,
              patience=5,
              optimizer='adamw',
              weight_decay=8.413267202596088e-05,
              embedding_dim=64,
              hidden_dims=(128,),
              dropout=0.5,
              seq_pooling='mean',
              n_classes=2,
              label_col='is_malware',
              dataloader_num_workers=0,
              dataloader_pin_memory=True,
              dataloader_persistent_workers=False,
              grad_scaler_max_norm=1.0)


In [7]:
all_trails_df.sort_values(
    by=["value", "number"], ascending=[False, True], inplace=True
)

all_trails_df

,number,value,datetime_start,datetime_complete,duration,params_batch_size,params_dropout,params_embedding_dim,params_hidden_dims_idx,params_max_learning_rate,params_weight_decay,state
80,80,0.973434,2025-06-22 14:56:51.007751,2025-06-22 14:57:34.772150,0 days 00:00:43.764399,64,0.5,64,3,0.006311,0.000084,COMPLETE
71,71,0.973380,2025-06-22 14:49:34.676567,2025-06-22 14:50:18.841917,0 days 00:00:44.165350,64,0.5,64,4,0.002183,0.000043,COMPLETE
84,84,0.972892,2025-06-22 14:59:41.690610,2025-06-22 15:00:25.458343,0 days 00:00:43.767733,64,0.5,64,4,0.004992,0.000080,COMPLETE
93,93,0.972892,2025-06-22 15:06:25.841823,2025-06-22 15:07:10.901604,0 days 00:00:45.059781,64,0.5,64,3,0.005468,0.000140,COMPLETE
23,23,0.972837,2025-06-22 11:23:19.814748,2025-06-22 11:24:05.650193,0 days 00:00:45.835445,64,0.5,64,3,0.005856,0.000161,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...
1,1,0.828909,2025-06-22 10:23:35.920028,2025-06-22 10:24:30.653731,0 days 00:00:54.733703,32,0.3,64,14,0.000030,0.000300,COMPLETE
5,5,0.801853,2025-06-22 10:48:57.538700,2025-06-22 10:49:52.828404,0 days 00:00:55.289704,32,0.2,64,0,0.000037,0.000010,COMPLETE
65,65,0.790164,2025-06-22 14:44:44.527855,2025-06-22 14:45:37.711430,0 days 00:00:53.183575,32,0.3,64,4,0.000012,0.000068,COMPLETE
8,8,0.770241,2025-06-22 11:06:11.765708,2025-06-22 11:08:17.447119,0 days 00:02:05.681411,16,0.2,128,14,0.000036,0.000015,COMPLETE
